# Histopathologic Cancer Detection with CNN Models

## Exploratory Data Analysis

Cancerous cells have at least one tumour cells in the center 32x32 region.

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pylab as plt
import seaborn as sns

from tifffile import imread

import tensorflow as tf
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, BatchNormalization, Activation, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

import os
#!pip install visualkeras
#import visualkeras
from skimage import draw

In [ ]:
#Loading training directory
train_df = pd.read_csv('/kaggle/input/histopathologic-cancer-detection/train_labels.csv')
train_df['label'] = train_df['label'].astype(str)
train_df['id'] = train_df['id'] + '.tif'
train_df.head()

In [ ]:
#Checking the labels, how many are negative and how many are positive 
sns.displot(data=train_df, x='label', hue='label')
train_df['label'].value_counts()

In [ ]:
#Loading the data with the images 

batch_size, im_size = 256, (96,96)

generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_data = generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'id', # filenames
    y_col = 'label', # labels
    directory = '/kaggle/input/histopathologic-cancer-detection/train',
    subset = 'training',
    class_mode = 'binary',
    batch_size = batch_size,
    target_size = im_size)

val_data = generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'id', # filenames
    y_col = 'label', # labels
    directory = '/kaggle/input/histopathologic-cancer-detection/train',
    subset = "validation",
    class_mode = 'binary',
    batch_size = batch_size,
    target_size = im_size)

In [ ]:
#PLotting the training graph function with accuracy and loss history 

def plot_hist(hist):
    plt.figure(figsize=(10,5))
    plt.subplot(121)
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history['val_accuracy'])
    plt.legend(["Accuracy","Validation Accuracy"])
    plt.ylabel("Accuracy")
    plt.xlabel("Epoch")
    plt.grid()
    plt.subplot(122)
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title("Model Evaluation")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.grid()
    plt.legend(["Loss","Validation Loss"])
    #plt.suptitle("Model Evaluation")
    plt.tight_layout()
    plt.show()

## VGG16 Backbone

In [ ]:
base_model = tf.keras.applications.VGG16(
    input_shape=(im_size[0],im_size[1],3), 
    include_top=False, 
    weights='imagenet'
)

![vgg16.png](attachment:66b20712-27f9-437e-ad9a-e3c97b55a636.png)

In [ ]:
#Original Model
np.random.seed(1)
tf.random.set_seed(1)
model_vgg16 = Sequential([
    base_model,
    Flatten(),    
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(8, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
], name='vgg16_backbone')
model_vgg16.summary()

In [ ]:
#Model 1
np.random.seed(1)
tf.random.set_seed(1)
model_vgg16 = Sequential([
    base_model,
    Flatten(),    
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(8, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, activation='sigmoid') 
], name='vgg16_backbone')
model_vgg16.summary()

In [ ]:
#Model 2
np.random.seed(1)
tf.random.set_seed(1)
model_vgg16 = Sequential([
    base_model,
    Flatten(),    
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, activation='sigmoid'),
    BatchNormalization()
], name='vgg16_backbone')
model_vgg16.summary()

## Training the model

In [ ]:
#Compiling code and declaring optimizer 
base_model.trainable = True
opt = tf.keras.optimizers.Adam(0.001)
#opt = tf.keras.optimizers.SGD(0.001)
model_vgg16.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

In [ ]:
model_vgg16.summary()

In [ ]:
%%time 
# Training 
hist = model_vgg16.fit(train_data, epochs = 9, validation_data = val_data, verbose=1)

In [ ]:
#Plotting Accuracies during training
plot_hist(hist)

## VGG19 Backbone

In [ ]:
base_model = tf.keras.applications.VGG19(
    input_shape=(im_size[0],im_size[1],3), 
    include_top=False, 
    weights='imagenet'
)

![vgg19.png](attachment:c7fa0523-b440-4291-904b-08f02c608c1f.png)

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)
#Model 2
model_vgg19 = Sequential([
    base_model,
    Flatten(),    
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, activation='sigmoid'),
    BatchNormalization()
], name='vgg19_backbone')

model_vgg19.summary()

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

#Original Model 
model_vgg19 = Sequential([
    base_model,
    Flatten(),    
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(8, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
], name='vgg19_backbone')

model_vgg19.summary()

In [ ]:
#Model 1
np.random.seed(1)
tf.random.set_seed(1)
model_vgg19 = Sequential([
    base_model,
    Flatten(),    
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(8, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, activation='sigmoid') 
], name='vgg19_backbone')
model_vgg19.summary()

In [ ]:
#Compiling model and declaring optimizer
opt = tf.keras.optimizers.Adam(0.001)
#opt = tf.keras.optimizers.SGD(0.001)
base_model.trainable = True
model_vgg19.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

In [ ]:
%%time 
# Training
hist = model_vgg19.fit(train_data, epochs = 7, validation_data = val_data, verbose=1)

In [ ]:
#Plotting Accuracies during training
plot_hist(hist)

## ResNet50 Backbone

In [ ]:
base_model = tf.keras.applications.ResNet50(
    input_shape=(im_size[0],im_size[1],3), 
    include_top=False, 
    weights='imagenet'
)

![resnet50.png](attachment:c6d2a377-c2dc-4dab-8dcf-283cbe6c5e67.png)

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

#Model 1
model_resnet50 = Sequential([
    base_model,
    Flatten(),    
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(8, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
], 'resnet50_backbone')

model_resnet50.summary()

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

#Model 2
model_resnet50 = Sequential([
    base_model,
    Flatten(),    
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, activation='sigmoid'),
    #BatchNormalization()
], 'resnet50_backbone')

model_resnet50.summary()

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

#Original Model 
model_resnet50 = Sequential([
    base_model,
    Flatten(),    
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(8, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
], name='resnet50_backbone')

model_resnet50.summary()

## Training

In [ ]:
#Compiling model and declaring optimizer 
base_model.trainable = True
#opt = tf.keras.optimizers.SGD(0.0003)
opt = tf.keras.optimizers.Adam(0.001)
model_resnet50.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

In [ ]:
%%time 
#remnoved batch normalization
#Training
hist = model_resnet50.fit(train_data, validation_data=val_data, epochs=8, verbose=1)

In [ ]:
#Plotting Accuracies during training
plot_hist(hist)

## Resnet 101

In [ ]:
base_model = tf.keras.applications.ResNet101(
    input_shape=(im_size[0],im_size[1],3), 
    include_top=False, 
    weights='imagenet'
)

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

#Model 1
model_resnet101 = Sequential([
    base_model,
    Flatten(),    
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(8, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
], 'resnet101_backbone')

model_resnet101.summary()

In [ ]:
# Compiling the model and declaring the optimizer 
base_model.trainable = True
opt = tf.keras.optimizers.Adam(0.001)
#opt = tf.keras.optimizers.SGD(0.001)
model_resnet101.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

#Model 2
model_resnet101 = Sequential([
    base_model,
    Flatten(),    
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, activation='sigmoid'),
    #BatchNormalization()
], 'resnet101_backbone')

model_resnet101.summary()

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

#Original Model 
model_resnet101 = Sequential([
    base_model,
    Flatten(),    
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(8, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
], name='resnet101_backbone')

model_resnet101.summary()

In [ ]:
%%time 
# Training
hist = model_resnet101.fit(train_data, validation_data=val_data, epochs=7, verbose=1)

In [ ]:
#Plotting Accuracies during training
plot_hist(hist)

## MobileNetV2

In [ ]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=(im_size[0],im_size[1],3), 
    include_top=False, 
    weights='imagenet'
)

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

#Model 2
model_mobilenetv2 = Sequential([
    base_model,
    Flatten(),    
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, activation='sigmoid'),
    #BatchNormalization()
], 'mobilenetv2_backbone')

model_mobilenetv2.summary()

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

#Original Model 
model_mobilenetv2 = Sequential([
    base_model,
    Flatten(),    
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(8, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
], name='mobilenetv2_backbone')

model_mobilenetv2.summary()

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

#Model 1
model_mobilenetv2 = Sequential([
    base_model,
    Flatten(),    
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(8, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
], 'mobilnetv2_backbone')

model_mobilenetv2.summary()

In [ ]:
#Declaring optimizer and compiling the model 
base_model.trainable = True
opt = tf.keras.optimizers.SGD(0.001)
#opt = tf.keras.optimizers.Adam(0.001)
model_mobilenetv2.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

In [ ]:
#Plotting Accuracies during training
plot_hist(hist)

In [ ]:
# Trainining
hist = model_mobilenetv2.fit(train_data, validation_data=val_data, epochs=7, verbose=1)

## MobileNet

In [ ]:
base_model = tf.keras.applications.mobilenet.MobileNet(
    input_shape=(im_size[0],im_size[1],3), 
    include_top=False, 
    weights='imagenet'
)

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

#Model 1
model_mobilenet = Sequential([
    base_model,
    Flatten(),    
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(8, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
], 'mobilnet_backbone')

model_mobilenet.summary()

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

#Original Model 
model_mobilenet = Sequential([
    base_model,
    Flatten(),    
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(8, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
], name='mobilenet_backbone')

model_mobilenet.summary()

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

#Model 2
model_mobilenet = Sequential([
    base_model,
    Flatten(),    
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, activation='sigmoid'),
    #BatchNormalization()
], 'mobilenet_backbone')

model_mobilenet.summary()

In [ ]:
#Declaring optimizer and compiling models 
base_model.trainable = True
opt = tf.keras.optimizers.SGD(0.001)
#opt = tf.keras.optimizers.Adam(0.001)
model_mobilenet.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

In [ ]:
#Plotting Accuracies during training
plot_hist(hist)

In [ ]:
# Training
hist = model_mobilenet.fit(train_data, validation_data=val_data, epochs=7, verbose=1)